## Rugby World Cup Venues - Travelling Salesman Problem

We have a file containing distances (in metres) and times (in seconds) to cycle between various stadiums in the UK, all of which have been used to host matches in the Rugby World Cup. The file is stored in "tab separated value" (tsv) format. We want to find the best way to travel between all the stadiums in the list. By "best", we mean either shortest distance, or shortest time.

<em>Acknowledgements: this notebook was inspired by work of Randy Olson; his code to find the shortest path for a road trip across the USA can be found at his Github page [here](http://nbviewer.ipython.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb). The first version of this NAG notebook was created by John Muddle.</em>

First of all we define the various stadiums:

In [1]:
all_stadiums = ["Brighton community stadium, UK", 
                "City of Manchester Stadium, Ashton New Road, Manchester",
                "Elland Road, UK",
                "King Power Stadium, Filbert Way, Leicester",
                "Kingsholm Stadium, Kingsholm Road, Gloucester",
                "Millennium Stadium, Westgate Street, Cardiff",
                "Sandy Park Stadium, Exeter",
                "St. James' Park, Barrack Road, Newcastle upon Tyne",
                "Stadium MK, Grafton Street, Bletchley",
                "The Stadium, Queen Elizabeth Olympic Park",
                "Twickenham Stadium, Whitton Road, Twickenham",
                "Villa Park, Trinity Road, Birmingham",
                "Wembley Stadium, Wembley"]

Then we load distance and time data from a file, using a routine from pandas, a Python data analysis library. The file is stored in "tab separated value" (tsv) format. The distances and times in the file were calculated using Google maps.

In [2]:
import pandas as pn
stadium_data = pn.read_csv("my-rugby-bike-stadiums-dist-dur.tsv", sep="\t")

Let's take a look at the contents of the file. Since we have n=13 stadiums, and the file needs to contain distances between every pair of stadiums, the file contains 12+11+10+...+1 = n*(n-1)/2 = 78 lines.

In [3]:
stadium_data

,stadium1,stadium2,distance_m,duration_s
0,"Brighton community stadium, UK","King Power Stadium, Filbert Way, Leicester",280176,52420
1,"Brighton community stadium, UK","Sandy Park Stadium, Exeter",313295,62141
2,"Brighton community stadium, UK","St. James' Park, Barrack Road, Newcastle upon ...",622834,116360
3,"Brighton community stadium, UK","Stadium MK, Grafton Street, Bletchley",184675,34481
4,"Brighton community stadium, UK","Twickenham Stadium, Whitton Road, Twickenham",106895,20637
...,...,...,...,...
73,"Wembley Stadium, Wembley","Sandy Park Stadium, Exeter",327375,62045
74,"Wembley Stadium, Wembley","St. James' Park, Barrack Road, Newcastle upon ...",512945,96107
75,"Wembley Stadium, Wembley","Stadium MK, Grafton Street, Bletchley",75540,13953
76,"Wembley Stadium, Wembley","Twickenham Stadium, Whitton Road, Twickenham",16002,3315


Create matrices containing distances and times, for input to the NAG function

In [4]:
import numpy as np
distance_matrix = np.zeros((len(all_stadiums), len(all_stadiums)))
duration_matrix = np.zeros((len(all_stadiums), len(all_stadiums)))
for index, row in stadium_data.iterrows():
    distance_matrix[all_stadiums.index(row['stadium1'])][all_stadiums.index(row['stadium2'])] = row['distance_m']
    distance_matrix[all_stadiums.index(row['stadium2'])][all_stadiums.index(row['stadium1'])] = row['distance_m']
    duration_matrix[all_stadiums.index(row['stadium1'])][all_stadiums.index(row['stadium2'])] = row['duration_s']
    duration_matrix[all_stadiums.index(row['stadium2'])][all_stadiums.index(row['stadium1'])] = row['duration_s']

Setup the NAG solver mip.tsp_simann

Function tsp_simann requires a randomly generated state to initalise; we use rand.init_repeat to define this state.

In [5]:
from naginterfaces.library import rand
statecomm = rand.init_repeat(genid=2, seed=[304950, 889934, 209094, 23423990], subid=53)

Find the optimal path using mip.tsp_simann. We call the function twice, once to find optimal distance, and once to find optimal time.

In [6]:
from naginterfaces.library import mip
soln_distance = mip.tsp_simann(dm=distance_matrix, bound=-1.0, targc=-1.0, statecomm=statecomm)
soln_duration = mip.tsp_simann(dm=duration_matrix, bound=-1.0, targc=-1.0, statecomm=statecomm)
print(soln_distance.path)
print(soln_duration.path)

[ 1  7  6  5 12  2  8  3  4  9 13 11 10]
[ 1 10 11 13  9  4  8  3  2 12  5  6  7]


The desired paths are stored in the integer list "path" of the tuple returned by mip.tsp_simann. We convert from the returned paths to the corresponding stadiums

In [7]:
optimal_path_distance = []
optimal_path_duration = []
for i in soln_distance.path:
    optimal_path_distance.append(all_stadiums[i-1])
for i in soln_duration.path:
    optimal_path_duration.append(all_stadiums[i-1])

And print the results

In [8]:
print('This is the shortest path by distance, of {:4.2f} kilometres:'.format(soln_distance.cost/1000))
print(optimal_path_distance)

This is the shortest path by distance, of 1793.38 kilometres:
['Brighton community stadium, UK', 'Sandy Park Stadium, Exeter', 'Millennium Stadium, Westgate Street, Cardiff', 'Kingsholm Stadium, Kingsholm Road, Gloucester', 'Villa Park, Trinity Road, Birmingham', 'City of Manchester Stadium, Ashton New Road, Manchester', "St. James' Park, Barrack Road, Newcastle upon Tyne", 'Elland Road, UK', 'King Power Stadium, Filbert Way, Leicester', 'Stadium MK, Grafton Street, Bletchley', 'Wembley Stadium, Wembley', 'Twickenham Stadium, Whitton Road, Twickenham', 'The Stadium, Queen Elizabeth Olympic Park']


In [9]:
print('This is the shortest path by time, of {:4.2f} hours:'.format(soln_duration.cost/3600))
print(optimal_path_duration)

This is the shortest path by time, of 96.06 hours:
['Brighton community stadium, UK', 'The Stadium, Queen Elizabeth Olympic Park', 'Twickenham Stadium, Whitton Road, Twickenham', 'Wembley Stadium, Wembley', 'Stadium MK, Grafton Street, Bletchley', 'King Power Stadium, Filbert Way, Leicester', "St. James' Park, Barrack Road, Newcastle upon Tyne", 'Elland Road, UK', 'City of Manchester Stadium, Ashton New Road, Manchester', 'Villa Park, Trinity Road, Birmingham', 'Kingsholm Stadium, Kingsholm Road, Gloucester', 'Millennium Stadium, Westgate Street, Cardiff', 'Sandy Park Stadium, Exeter']
